# Assignment 7

### Part A

In [13]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [14]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })
portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [15]:
def expected_return(P, rf, mkt_rp):
    df = P.copy(deep=True)
    df["Weight"] = df["Amount Invested"] / sum(df["Amount Invested"])
    beta = sum(df["Weight"] * df["Beta"])
    return rf + beta * mkt_rp

In [16]:
expected_return(portfolio, 0.02, 0.05)

0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [17]:
def volatility(P, cor):
    df = P.copy(deep=True)
    df["Weight"] = df["Amount Invested"] / sum(df["Amount Invested"])
    var = 0
    n = len(df.index)
    for i in range(n):
        var += (df["Weight"].iloc[i] ** 2) * (df["Volatility"].iloc[i] ** 2)
        for j in range(n):
            if i != j:
                cov = cor * (((df["Volatility"].iloc[i] ** 2) * (df["Volatility"].iloc[j] ** 2)) ** 0.5)
                var += df["Weight"].iloc[i] * df["Weight"].iloc[j] * cov
    return var ** 0.5

In [18]:
volatility(portfolio, -0.2)

0.24083189157584592

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [19]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    return rf + (vol / mkt_vol) * mkt_rp

In [20]:
expected_return2(0.02, 0.05, 0.16, 0.2408)

0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [21]:
def CAPM_beta(return_path, factors_path):
    df_return = pd.read_csv(return_path)
    df_factors = pd.read_csv(factors_path)

    df = df_return.copy(deep=True)
    df["date"] = df["year"] * 100 + df["month"]
    df = pd.merge(df, df_factors, how="left", on="date")
    df["Mkt-RF"] = df["Mkt-RF"] / 100
    df["RF"] = df["RF"] / 100
    df["RET - RF"] = df["RET"] - df["RF"]

    df_1 = df[["CUSIP", "RET"]].groupby("CUSIP", sort=False).mean().reset_index()

    df_2 = df.groupby("CUSIP", sort=False)[["RET - RF", "Mkt-RF"]].apply(lambda x: x.cov().iloc[0, 1] / x["Mkt-RF"].var())
    df_2 = df_2.reset_index()

    df_3 = pd.merge(df_1, df_2, how="left", on="CUSIP")

    output = pd.DataFrame(data = {
        "CUSIP" : df_3["CUSIP"],
        "EXP_RETURN" : df_3["RET"],
        "BETA" : df_3[0],
    })
    
    return output

In [22]:
data_capm = CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV")

print(data_capm)

         CUSIP  EXP_RETURN      BETA
0        36020    0.024084  0.854739
1        36110    0.012326  1.301644
2        37520    0.013264  1.481594
3     00081T10    0.016252  2.591073
4        88630    0.020888  1.765291
...        ...         ...       ...
3220  47836610    0.013981  1.161753
3221  47916710    0.010823  0.868461
3222  47916920    0.011985  1.331554
3223  47936810    0.012286  1.347749
3224  48020610    0.017315  1.597907

[3225 rows x 3 columns]


Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [23]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    return np.polyfit(data["BETA"], data["EXP_RETURN"], 1)[1]

In [24]:
slope(data_capm)

0.007981588120535638